In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bertmodel/config.json
/kaggle/input/bertmodel/README.md
/kaggle/input/bertmodel/tokenizer.json
/kaggle/input/bertmodel/tokenizer_config.json
/kaggle/input/bertmodel/pytorch_model.bin
/kaggle/input/bertmodel/model.safetensors
/kaggle/input/bertmodel/special_tokens_map.json
/kaggle/input/bertmodel/vocab.txt
/kaggle/input/selected-data-csv/selected_data.csv
/kaggle/input/gujarati-filtered-data/filtered_data.csv


In [ ]:
import pandas as pd
import random

# Load the dataset
dataset = pd.read_csv('/kaggle/input/gujarati-filtered-data/filtered_data.csv')


# Display the selected data
print(dataset)
print(dataset.shape)


print(test_dataset)

In [ ]:
pip install transformers


In [ ]:
# Calculate accuracy
#         _,predicted_indices = torch.max(logits, 2)
#         print("predicted_indices",predicted_indices)
#         print("target_indices",target_indices.view)
#         correct_predictions += torch.sum(torch.eq(predicted_indices, target_indices.view(-1, 1))).item()
#         total_predictions += target_indices.size(0)
        
#         print("total_predictions",total_predictions)
#         print("target_indices_size",len(target_indices))

#          # Calculate precision
#         predicted_word_indices = predicted_indices.squeeze().tolist()  # Convert to list
#         target_word_indices = target_indices.squeeze().tolist()  # Convert to list
#         total_target_occurrences += len(target_word_indices)

#         for predicted_idx in predicted_word_indices:
#             if predicted_idx in target_word_indices:
#                 correct_predictions += 1

#         precision = correct_predictions / (total_predictions + 1e-8) # Avoid division by zero
#         print("Precision",precision)

# def validate(model, dataloader, device):
#     model.eval()
#     total_loss = 0
#     correct_predictions = 0
#     total_predictions = 0

#     with torch.no_grad():
#         for batch in tqdm(dataloader, desc="Validation"):
#             input_ids = batch['input_ids'].to(device)
#             attention_mask = batch['attention_mask'].to(device)
#             target_indices = batch['target_indices'].squeeze().to(device)

#             outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#             logits = outputs.logits

#             # Calculate loss for target word positions
#             loss = torch.mean(logits[0, target_indices])
#             total_loss += loss.item()
#             print(total_loss)

#             # Make predictions and track accuracy
#             predicted_indices = logits.argmax(dim=2)
#             correct_predictions += torch.sum(predicted_indices == target_indices.unsqueeze(-1)).item()
#             total_predictions += target_indices.numel()

#     avg_loss = total_loss / len(dataloader)
#     accuracy = correct_predictions / total_predictions

#     print(f"Validation Avg Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")


In [5]:
import pandas as pd

# Load the dataset
dataset = pd.read_csv('/kaggle/input/gujarati-filtered-data/filtered_data.csv')

# dataset = dataset[0:15]

dataset['word_count'] = dataset['text'].apply(lambda x: len(x.split()))
dataset = dataset[(dataset['word_count'] >= 3) & (dataset['word_count'] < 63)]

# Display the filtered data and its shape
print(dataset)
print(dataset.shape)

                                                    text target_word  \
0      1 એપ્રિલ 2007ના દિવસે નોકિયાના નેટવર્ક્સ બિઝને...        સાથે   
1      મહાવીર સ્વામી, છેલ્લા તિર્થંકર, ક્ષત્રિય કુળમા...     વર્ણનાં   
2      બુદ્ધિપ્રકાશ ‍ ગુજરાતી ભાષાનું ગુજરાત વિદ્યા સ...     ગુજરાતી   
3                               ગુજરાત અને ભારતમાં સ્થાન         અને   
4                               ગુજરાત અને ભારતમાં સ્થાન       સ્થાન   
...                                                  ...         ...   
40727  આ જિલ્લો ૩૮૫૦ ચોરસ કિલોમીટર જેટલો વિસ્તાર ધરાવ...   જિલ્લામાં   
40728  સાલેજ ભારત દેશના પશ્ચિમ ભાગમાં આવેલા ગુજરાત રા...        જેવી   
40730  3,061 metres જેટલી લંબાઈનો માર્ગ ધરાવતો આ સેતુ...           આ   
40735  હવા પ્રદુષણ મોટેભાગે વધુ વસ્તીવાળા શહેરી વિસ્ત...     દેશોમાં   
40736  આઉટસીડ ઈન્સીડ માં લેસબિયન, ગે અને બાયસેક્સ્યુઅ...       જાતિય   

       word_count  
0              34  
1              37  
2              14  
3               4  
4               4  
...           .

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, BertForMaskedLM
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

# Define dataset class
class GujaratiWordSpottingDataset(Dataset):
    def __init__(self, texts, target_words, tokenizer, max_length=128):
        self.texts = texts
        self.target_words = target_words
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target_word = str(self.target_words[idx])

        # Tokenize inputs
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )

        # Tokenize target word
        target_token = self.tokenizer.tokenize(target_word)

        target_indices = []
        if target_token:  # Check if target_token is not empty
            for i in range(len(inputs['input_ids'][0])):
                token_str = self.tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][i].item())
                if token_str == target_token[0]:
                    target_indices.append(i)
        else:
            # If target_token is empty, return empty tensors
            return {
                'input_ids': torch.tensor([]),
                'attention_mask': torch.tensor([]),
                'target_indices': torch.tensor([])
            }

        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0),
            'target_indices': torch.tensor(target_indices)
        }


# Load dataset
texts = dataset['text'].tolist()
target_words = dataset['target_word'].tolist()

# Define paths to the downloaded model and tokenizer
model_path = "/kaggle/input/bertmodel"
tokenizer_path = "/kaggle/input/bertmodel"

# Load the tokenizer and model from the downloaded files
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, use_fast=False)
model = BertForMaskedLM.from_pretrained(model_path)

# Prepare dataset and dataloader
dataset = GujaratiWordSpottingDataset(texts, target_words, tokenizer)
train_dataset, val_dataset = train_test_split(dataset, test_size=0.2, random_state=42)  # Split dataset into train and validation
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001)

# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

for epoch in range(10):
    total_loss = 0

    # Use tqdm for progress reporting
    with tqdm(train_dataloader, desc=f"Epoch {epoch + 1}", unit="batch") as tepoch:
        for batch_idx, batch in enumerate(tepoch):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            target_indices = batch['target_indices'].squeeze().to(device)

            if target_indices.dim() == 0 or target_indices.numel() == 0:
                continue  # Skip this batch

            if target_indices.dim() == 0:
                # Ensure target_indices has at least one dimension
                target_indices = target_indices.unsqueeze(0)  
            optimizer.zero_grad()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            # Calculate loss for each target index
            loss = 0
            for idx in target_indices:
                loss += F.cross_entropy(logits[:, idx, :], idx.unsqueeze(0).to(device))

            # Backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Update tqdm description
            tepoch.set_postfix(loss=loss.item())

    # Calculate average 
    avg_loss = total_loss / len(train_dataloader)

    print(f"Epoch {epoch + 1}, Train Avg Loss: {avg_loss:.4f}")
      

# Save the model and tokenizer
model.save_pretrained("gujarati_word_spotting_model")
tokenizer.save_pretrained("gujarati_word_spotting_model")
